In [6]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

C:\Users\Saleem\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
training_set = np.array(pd.read_csv("ml-100k/u1.base", delimiter="\t"), dtype="int")
training_set

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [8]:
test_set = np.array(pd.read_csv("ml-100k/u1.test", delimiter="\t"), dtype="int")
test_set

array([[        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       ...,
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]])

In [9]:
nb_users = len(set(np.concatenate((training_set[:, 0], test_set[:, 0]))))
nb_movies = len(set(np.concatenate((training_set[:, 1], test_set[:, 1]))))

print(nb_users, nb_movies)

943 1682


In [10]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:, 1][data[:, 0] == id_users]
        id_ratings = data[:, 2][data[:, 0] == id_users]
        ratings_arr = np.zeros(nb_movies)
        ratings_arr[id_movies - 1] = id_ratings
        new_data.append(list(ratings_arr))
    return new_data

In [11]:
training_set = convert(training_set)
test_set = convert(test_set)

In [12]:
training_set[0][:3]

[0.0, 3.0, 4.0]

In [13]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [14]:
print(training_set)
print(test_set)

tensor([[0., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [15]:
class SAE(nn.Module):

    def __init__(self, nb_movies):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 128)
        self.fc2 = nn.Linear(128, 20)
        self.fc3 = nn.Linear(20, 128)
        self.fc4 = nn.Linear(128, nb_movies)
        self.activation = nn.Sigmoid()


    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x


In [16]:
sae = SAE(nb_movies)
criterion = nn.MSELoss()
optimizer = optim.Adam(sae.parameters(), lr=0.001, weight_decay=0.5)


In [47]:
nb_epochs = 200

for epoch in range(1, nb_epochs + 1):
    train_loss = 0
    s = 0.

    for user_id in range(nb_users):
        input = Variable(training_set[user_id]).unsqueeze(0)
        target = input.detach()

        if torch.sum(target.data > 0) > 0:

            output = sae(input)
            target.require_grad = False
            # output[target == 0] = 0
            output = torch.where(target == 0, 0, output)

            loss = criterion(output, target)
            mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
            train_loss += np.sqrt(loss.item() * mean_corrector)
            # train_loss += np.sqrt(loss.item())
            s += 1

            optimizer.zero_grad()  # when adding this line the loss is stuck on 3.7 investigate this

            loss.backward()
            optimizer.step()

    print(f"epoch {epoch} train_loss: {train_loss / s}")





epoch 0 train_loss: 1.3957177271751158
epoch 1 train_loss: 1.097887222400596
epoch 2 train_loss: 1.0657431184741055


KeyboardInterrupt: 

In [37]:

test_loss = 0
s = 0.

for user_id in range(nb_users):
    input = Variable(training_set[user_id]).unsqueeze(0)  # same as in boltzmann machine
    target = Variable(test_set[user_id]).unsqueeze(0)

    if torch.sum(target.data > 0) > 0:

        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0

        # optimizer.zero_grad()  # when adding this line the loss is stuck on 3.7 investigate this

        loss = criterion(output, target)
        mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.item() * mean_corrector)
        s += 1


print(f"test_loss: {test_loss / s}")

test_loss: 0.9491299440159031


In [38]:
nb_epochs = 200

for epoch in range(1, nb_epochs + 1):
    train_loss = 0
    s = 0.

    for user_id in range(nb_users):
        input = Variable(training_set[user_id]).unsqueeze(0)
        target = input.clone()

        if torch.sum(target.data > 0) > 0:

            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0

            optimizer.zero_grad()  # when adding this line the loss is stuck on 3.7 investigate this

            loss = criterion(output, target)
            mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
            train_loss += np.sqrt(loss.item() * mean_corrector)
            s += 1

            loss.backward()
            optimizer.step()

    print(f"epoch {epoch} train_loss: {train_loss / s}")



epoch 1 train_loss: 3.5290706689602773


KeyboardInterrupt: 